In [71]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float, DateTime, VARCHAR, Interval
from sqlalchemy.orm import Session

In [72]:
#Creating Dataframes from Cleaned CSVs
cleanmeasure = pd.read_csv('clean_hawaii_measurements.csv')
cleanstation = pd.read_csv('clean_hawaii_statioins.csv')

In [73]:
#Check if worked
cleanmeasure.head()

,STATION,DATE,PRCP,TOBS
0,USC00519397,1/1/10,0.08,65
1,USC00519397,1/2/10,0.00,63
2,USC00519397,1/3/10,0.00,74
3,USC00519397,1/4/10,0.00,76
4,USC00519397,1/7/10,0.06,70


In [98]:
#create isodate in pandas
cleanmeasure['ISODATE'] = pd.to_datetime(cleanmeasure['DATE'], format='%m/%d/%y')
cleanmeasure.head()

,STATION,DATE,PRCP,TOBS,ISODATE
0,USC00519397,1/1/10,0.08,65,2010-01-01
1,USC00519397,1/2/10,0.00,63,2010-01-02
2,USC00519397,1/3/10,0.00,74,2010-01-03
3,USC00519397,1/4/10,0.00,76,2010-01-04
4,USC00519397,1/7/10,0.06,70,2010-01-07


In [75]:
#Check if worked
cleanstation.head()

,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [76]:
#Creating a database called Hawaii.
engine = create_engine("sqlite:///hawaii.sqlite")

In [77]:
#Connecting to Database
conn = engine.connect()

In [78]:
#Declaring a base
Base = declarative_base()

In [79]:
#Creating Measurement Table
#DateTime is only way the IsoDate would be parsed.
class Measurement(Base):
    __tablename__ = 'Measurement'
    
    id = Column(Integer, primary_key = True)
    STATION = Column(VARCHAR)
    DATE = Column(Text)
    PRCP = Column(Integer)
    TOBS = Column(Integer)
    ISODATE = Column(DateTime)
    
    def __repr__(self):
        return f"id={self.id}, Date={self.Date}"

In [80]:
#Creating Station Table
class Station(Base):
    
    __tablename__ = 'Station'
    
    id = Column(Integer, primary_key = True)
    STATION = Column(VARCHAR)
    NAME = Column(VARCHAR)
    LATITUDE = Column(Float)
    LONGITUDE = Column(Float)
    ELEVATION = Column(Float)
    
    def __repr__(self):
        return f"id={self.id}, Name={self.Name}"

In [81]:
#Create Station and Measurement Table in Database
Base.metadata.create_all(engine)

In [82]:
#Creating JSONIFIED data out of Measurement Data
data = cleanmeasure.to_dict(orient='records')

In [83]:
#Check if above worked
print(data)

[{'STATION': 'USC00519397', 'DATE': '1/1/10', 'PRCP': 0.08, 'TOBS': 65, 'ISODATE': Timestamp('2010-01-01 00:00:00')}, {'STATION': 'USC00519397', 'DATE': '1/2/10', 'PRCP': 0.0, 'TOBS': 63, 'ISODATE': Timestamp('2010-01-02 00:00:00')}, {'STATION': 'USC00519397', 'DATE': '1/3/10', 'PRCP': 0.0, 'TOBS': 74, 'ISODATE': Timestamp('2010-01-03 00:00:00')}, {'STATION': 'USC00519397', 'DATE': '1/4/10', 'PRCP': 0.0, 'TOBS': 76, 'ISODATE': Timestamp('2010-01-04 00:00:00')}, {'STATION': 'USC00519397', 'DATE': '1/7/10', 'PRCP': 0.06, 'TOBS': 70, 'ISODATE': Timestamp('2010-01-07 00:00:00')}, {'STATION': 'USC00519397', 'DATE': '1/8/10', 'PRCP': 0.0, 'TOBS': 64, 'ISODATE': Timestamp('2010-01-08 00:00:00')}, {'STATION': 'USC00519397', 'DATE': '1/9/10', 'PRCP': 0.0, 'TOBS': 68, 'ISODATE': Timestamp('2010-01-09 00:00:00')}, {'STATION': 'USC00519397', 'DATE': '1/10/10', 'PRCP': 0.0, 'TOBS': 73, 'ISODATE': Timestamp('2010-01-10 00:00:00')}, {'STATION': 'USC00519397', 'DATE': '1/11/10', 'PRCP': 0.01, 'TOBS': 

In [84]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [85]:
# Save the reference to the `Measurement` table as a variable called `table`
table = sqlalchemy.Table('Measurement', metadata, autoload=True)

In [86]:
#Remove pre-existing data
conn.execute(table.delete())

In [87]:
#Insert Data into table
conn.execute(table.insert(), data)

In [88]:
#Test to see if it worked
conn.execute("select * from Measurement limit 10").fetchall()
#Note if Schema is screwed up - delete the sqlite file and start over again.

[(1, 'USC00519397', '1/1/10', 0.08, 65, '2010-01-01 00:00:00.000000'),
 (2, 'USC00519397', '1/2/10', 0, 63, '2010-01-02 00:00:00.000000'),
 (3, 'USC00519397', '1/3/10', 0, 74, '2010-01-03 00:00:00.000000'),
 (4, 'USC00519397', '1/4/10', 0, 76, '2010-01-04 00:00:00.000000'),
 (5, 'USC00519397', '1/7/10', 0.06, 70, '2010-01-07 00:00:00.000000'),
 (6, 'USC00519397', '1/8/10', 0, 64, '2010-01-08 00:00:00.000000'),
 (7, 'USC00519397', '1/9/10', 0, 68, '2010-01-09 00:00:00.000000'),
 (8, 'USC00519397', '1/10/10', 0, 73, '2010-01-10 00:00:00.000000'),
 (9, 'USC00519397', '1/11/10', 0.01, 64, '2010-01-11 00:00:00.000000'),
 (10, 'USC00519397', '1/12/10', 0, 61, '2010-01-12 00:00:00.000000')]

In [89]:
#Creating JSONIFIED data out of Station Data
data = cleanstation.to_dict(orient='records')

In [90]:
#Check if above worked
print(data)

[{'STATION': 'USC00519397', 'NAME': 'WAIKIKI 717.2, HI US', 'LATITUDE': 21.2716, 'LONGITUDE': -157.8168, 'ELEVATION': 3.0}, {'STATION': 'USC00513117', 'NAME': 'KANEOHE 838.1, HI US', 'LATITUDE': 21.4234, 'LONGITUDE': -157.8015, 'ELEVATION': 14.6}, {'STATION': 'USC00514830', 'NAME': 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 'LATITUDE': 21.5213, 'LONGITUDE': -157.8374, 'ELEVATION': 7.0}, {'STATION': 'USC00517948', 'NAME': 'PEARL CITY, HI US', 'LATITUDE': 21.3934, 'LONGITUDE': -157.9751, 'ELEVATION': 11.9}, {'STATION': 'USC00518838', 'NAME': 'UPPER WAHIAWA 874.3, HI US', 'LATITUDE': 21.4992, 'LONGITUDE': -158.0111, 'ELEVATION': 306.6}, {'STATION': 'USC00519523', 'NAME': 'WAIMANALO EXPERIMENTAL FARM, HI US', 'LATITUDE': 21.33556, 'LONGITUDE': -157.71139, 'ELEVATION': 19.5}, {'STATION': 'USC00519281', 'NAME': 'WAIHEE 837.5, HI US', 'LATITUDE': 21.45167, 'LONGITUDE': -157.84888999999995, 'ELEVATION': 32.9}, {'STATION': 'USC00511918', 'NAME': 'HONOLULU OBSERVATORY 702.2, HI US', 'LATITUDE': 2

In [91]:
# Save the reference to the `Station` table as a variable called `table`
table = sqlalchemy.Table('Station', metadata, autoload=True)

In [92]:
#Remove pre-existing data
conn.execute(table.delete())

In [93]:
# session = Session(bind=engine)

In [94]:
conn.execute(table.insert(), data)

In [95]:
conn.execute("select * from Station limit 10").fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6),
 (6, 'USC00519523', 'WAIMANALO EXPERIMENTAL FARM, HI US', 21.33556, -157.71139, 19.5),
 (7, 'USC00519281', 'WAIHEE 837.5, HI US', 21.45167, -157.84888999999995, 32.9),
 (8, 'USC00511918', 'HONOLULU OBSERVATORY 702.2, HI US', 21.3152, -157.9992, 0.9),
 (9, 'USC00516128', 'MANOA LYON ARBO 785.2, HI US', 21.3331, -157.8025, 152.4)]

In [96]:
#Note for above data column names and class Table names must exactly match. Otherwise the data won't be entered in.